In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from scipy.optimize import curve_fit
from scipy.stats import norm

In [ ]:
train_n_batches = 20

df_games = pd.concat((
    pd.read_parquet(f"parsed/batch_{i}_games.parquet")
    for i in range(1, train_n_batches + 1)
))

df_moves = pd.concat((
    pd.read_parquet(f"parsed/batch_{i}_moves.parquet")
    for i in range(1, train_n_batches + 1)
))

In [ ]:
# --- do not delete

a = np.concatenate([df_games["WhiteElo"].values, df_games["BlackElo"].values])
x = np.arange(a.min(), a.max(), 50)
y = norm(a.mean(), a.std()).pdf(x)

fig = px.line(
    x=x, y=y,
    template="plotly_white",
)
fig.data[0].mode="lines+markers"
fig.data[0].line.color="#008C45"
fig.data[0].line.width=4*2
fig.data[0].marker.size=10*2

fig.update_xaxes(title="")
fig.update_yaxes(title="")
# fig.update_layout(height=1080, width=1920, font_size=28*2, font_family="Consolas")
fig.show()

# fig.write_image("presentation/images/elo_hist.png", scale=2)

In [ ]:
# df_games["Result"] = df_games["Result"].map({
#     "1-0": 1,
#     "0-1": -1,
#     "1/2-1/2": 0
# }).fillna(0)

df_games["Result"] = df_games["Result"].map({
    "1-0": 1,
    "0-1": 0,
    "1/2-1/2": 0.50
}).fillna(0.50)

In [ ]:
df_moves = df_moves.merge(
    df_games[["GameId", "Result"]],
    on="GameId"
)

**MateIn**

In [ ]:
df_moves["MateIn"] = df_moves["Eval"].where(
    df_moves["Eval"].str.startswith("#"),
    "#0"
)
df_moves["MateIn"] = df_moves["MateIn"].str[1:].astype(float)
df_moves["MateIn"] = df_moves["MateIn"].clip(-30, 30)

In [ ]:
b = df_moves.groupby("MateIn", as_index=False).agg({"Result": "mean"})
b = b[ b["MateIn"] > 0 ]

In [ ]:
def inverse(x, a, b, c):
    return a/(x-b) + c

p0 = [1, 0, 0]

popt, _ = curve_fit(inverse, b["MateIn"], b["Result"], p0, method="lm")

In [ ]:
list(popt.round(6))

In [ ]:
fig = px.line(
    x=b["MateIn"].values,
    y=[b["Result"].values, inverse(b["MateIn"].values, *popt)]
)

# fig.data[1].line.color="black"
# fig.data[1].line.dash="dash"
# fig.data[0].marker.color="green"

fig.update_layout(height=650)

# fig.update_layout(xaxis_range=[0, 31])

fig.show()

**Evals**

In [ ]:
df_moves["EvalCentipawn"] = pd.to_numeric(df_moves["Eval"], errors="coerce").multiply(100).round()
df_moves["EvalCentipawn"] = df_moves["EvalCentipawn"].clip(-2000, 2000)

In [ ]:
a = df_moves.groupby(
    df_moves["EvalCentipawn"] // 50 * 50 + 25
).agg({"Result": "mean"}).squeeze()

In [ ]:
a

In [ ]:
def sigmoid(x, a1, a2, a3, a4):
    s = 1 + np.exp( -a3*(x-a2) )
    y = a1 / s + a4
    return y

p0 = [2, -8, 0.003, -0.8]

popt, _ = curve_fit(sigmoid, a.index, a.values, p0, method="lm")

In [ ]:
list(popt.round(6))

In [ ]:
fig = px.line(
    x=a.index / 100,
    y=sigmoid(a.index, *popt),
    template="plotly_white"
)

# fig.add_traces(px.scatter(
#     x=a.index / 100,
#     y=a.values
# ).data[0])

fig.data[0].mode="lines"
fig.data[0].line.color="#008C45"
# fig.data[0].line.width=4*2
# fig.data[0].marker.size=10*2

fig.update_xaxes(title="")
fig.update_yaxes(title="")
# fig.update_layout(height=1080, width=1920, font_size=28*2, font_family="Consolas")
fig.show()

# fig.write_image("presentation/images/winodds.png", scale=2)